In [1]:
import sys

sys.path.append("..")

In [2]:
import torch

In [3]:
torch.manual_seed(42)

In [4]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

## How to Use

### Prepare Instruments

We consider a `BrownianStock`, which is a stock following the geometric Brownian motion, and a `EuropeanOption` which is contingent on it.

We assume that the stock has a transaction cost of 1 basis point.

In [5]:
from pfhedge.instruments import BrownianStock
from pfhedge.instruments import EuropeanOption

stock = BrownianStock(cost=1e-4)
deriv = EuropeanOption(stock).to(DEVICE)

fail to import apex_C: apex was not installed or installed without --cpp_ext.
fail to import amp_C: apex was not installed or installed without --cpp_ext.


In [6]:
deriv

EuropeanOption(BrownianStock(...), maturity=8.22e-02)

### Create Your Hedger

We here use a multi-layer perceptron as our model.

In [ ]:
from pfhedge import Hedger
from pfhedge.nn import MultiLayerPerceptron

model = MultiLayerPerceptron()
hedger = Hedger(model, features=["log_moneyness", "expiry_time", "volatility", "prev_hedge"])
hedger = hedger.to(DEVICE)

The `hedger` is also a [`Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module).

In [8]:
hedger

Hedger(
  features=['log_moneyness', 'expiry_time', 'volatility', 'prev_hedge'],
  (model): MultiLayerPerceptron(
    (0): LazyLinear(in_features=None, out_features=32, bias=True)
    (1): ReLU()
    (2): LazyLinear(in_features=None, out_features=32, bias=True)
    (3): ReLU()
    (4): LazyLinear(in_features=None, out_features=32, bias=True)
    (5): ReLU()
    (6): LazyLinear(in_features=None, out_features=32, bias=True)
    (7): ReLU()
    (8): LazyLinear(in_features=None, out_features=1, bias=True)
    (9): Identity()
  )
  (criterion): EntropicRiskMeasure()
)

In [9]:
history = hedger.fit(deriv, n_epochs=200, n_paths=10000)

Loss=2.21596e-02:  96%|█████████▋| 193/200 [01:07<00:02,  2.84it/s]


KeyboardInterrupt: 

In [ ]:
price = hedger.price(deriv)
price

## More Examples

### Black-Scholes' Delta-Hedging Strategy

In [ ]:
from pfhedge import Hedger
from pfhedge.nn import BlackScholes

model = BlackScholes(deriv)
hedger = Hedger(model, model.features()).to(DEVICE)

In [ ]:
hedger

In [ ]:
price = hedger.price(deriv)
price

### Whalley-Wilmott's Asymptotically Optimal Strategy for Small Costs

In [ ]:
from pfhedge import Hedger
from pfhedge.nn import WhalleyWilmott

model = WhalleyWilmott(deriv)
hedger = Hedger(model, model.features()).to(DEVICE)

In [ ]:
price = hedger.price(deriv)
price

### Your Own Module

In [ ]:
import torch
import torch.nn.functional as fn
from pfhedge.nn import BlackScholes
from pfhedge.nn import Clamp
from pfhedge.nn import MultiLayerPerceptron


class NoTransactionBandNet(torch.nn.Module):
    def __init__(self, derivative):
        super().__init__()

        self.delta = BlackScholes(derivative)
        self.mlp = MultiLayerPerceptron(out_features=2)
        self.clamp = Clamp()

    def features(self):
        return self.delta.features() + ["prev_hedge"]

    def forward(self, x):
        prev_hedge = x[:, [-1]]

        delta = self.delta(x[:, :-1])
        width = self.mlp(x[:, :-1])

        lower = delta - fn.leaky_relu(width[:, [0]])
        upper = delta + fn.leaky_relu(width[:, [1]])

        return self.clamp(prev_hedge, min_value=lower, max_value=upper)

In [ ]:
model = NoTransactionBandNet(deriv)
hedger = Hedger(model, model.features()).to(DEVICE)

In [ ]:
history = hedger.fit(deriv, n_epochs=200, n_paths=10000)